# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
cities = pd.read_csv("output_data/cities.csv")
cities.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.12,-134.97,72.39,81,100,14.23,PF,1596005309
1,puerto varas,-41.32,-72.99,30.20,100,90,2.24,CL,1596005309
2,nabire,-3.37,135.48,83.71,72,100,1.70,ID,1596005310
3,buin,-33.73,-70.75,45.00,86,0,3.36,CL,1596005310
4,albany,42.60,-73.97,64.00,69,0,3.15,US,1596005033


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
gmaps.configure(api_key=g_key)

In [7]:
location = cities[["Lat","Lng"]]
weight = cities["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights=weight)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
i_cities = cities.loc[(cities["Cloudiness"]<=10)&(cities["Humidity"]>=80)&(cities["Max Temp"]>70)&(cities["Max Temp"]<=88)]
i_cities = i_cities.reset_index().drop(columns="index")
i_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,curuca,-0.73,-47.85,73.27,94,1,4.38,BR,1596005340
1,raymondville,26.48,-97.78,80.01,88,1,2.30,US,1596005341
2,mayo,38.89,-76.51,79.00,88,1,3.36,US,1596005342
3,lagoa,39.05,-27.98,73.08,90,0,17.36,PT,1596005362
4,porto walter,-8.27,-72.74,72.09,82,7,2.26,BR,1596005387
5,manaure,11.78,-72.44,80.10,83,2,8.55,CO,1596005393
6,houma,29.60,-90.72,75.99,100,1,5.39,US,1596005278
7,benjamin constant,-4.38,-70.03,71.67,97,7,1.63,BR,1596005408
8,cockburn town,21.46,-71.14,83.21,82,5,22.44,TC,1596005415
9,cayenne,4.93,-52.33,75.20,100,0,10.80,GF,1596005428


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = i_cities.loc[:,["City","Country"]]
hotel_df["Hotel Name"]=""
hotel_df["Lat"]=""
hotel_df["Lng"]=""

In [17]:
target_radius = 5000
target_type = "hotel"
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in i_cities.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index,"Hotel Name"] = response["results"][0]["name"]
        hotel_df.loc[index,"Lat"]=response["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index,"Lng"]=response["results"][0]["geometry"]["location"]["lng"]
    except IndexError:
        print(f"Missing {row['City']} field/result... skipping.")

In [18]:
hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!="")]
hotel_df

,City,Country,Hotel Name,Lat,Lng
0,curuca,BR,Curuçá,-0.733568,-47.8519
1,raymondville,US,Raymondville,26.4815,-97.7831
2,mayo,US,Mayo,38.8876,-76.5119
3,lagoa,PT,Santa Cruz Da Graciosa,39.0863,-28.0157
4,porto walter,BR,Porto Walter,-8.26361,-72.7543
5,manaure,CO,Manaure,11.7731,-72.4459
6,houma,US,Houma,29.5958,-90.7195
7,benjamin constant,BR,Benjamin Constant,-4.37599,-70.0302
8,cockburn town,TC,Cockburn Town,21.4675,-71.1389
9,cayenne,GF,Cayenne,4.92242,-52.3135


In [19]:
hotel_df

,City,Country,Hotel Name,Lat,Lng
0,curuca,BR,Curuçá,-0.733568,-47.8519
1,raymondville,US,Raymondville,26.4815,-97.7831
2,mayo,US,Mayo,38.8876,-76.5119
3,lagoa,PT,Santa Cruz Da Graciosa,39.0863,-28.0157
4,porto walter,BR,Porto Walter,-8.26361,-72.7543
5,manaure,CO,Manaure,11.7731,-72.4459
6,houma,US,Houma,29.5958,-90.7195
7,benjamin constant,BR,Benjamin Constant,-4.37599,-70.0302
8,cockburn town,TC,Cockburn Town,21.4675,-71.1389
9,cayenne,GF,Cayenne,4.92242,-52.3135


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)

# Display figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))